In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import csv
import collections
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(20,13)})
plt.style.use('ggplot')

In [2]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

In [3]:
# Defining an array which holds the names of the components of the events

EVENTS = ['target', 'source', 'action', 'location', 'date', 'action_type', 'target_group', 'action_group', 'source_group']

In [7]:
path_input = '/Users/shahreenshahjahanpsyche/Desktop/SATP-Event-Classification/Data_Annotated/master.csv'
df = pd.read_csv(path_input, header=[0])
df.shape

(4500, 14)

In [13]:
"""

    This function processes the string to arraytype for each row and also fills up the null values
    
    INPUT - df - Dataframe
            col - Name of the column. The column is a string type column
    

"""
def process_column(df, col):
    df.fillna("NotSpecified", inplace=True)
    df[col] = df[col].str.strip('[]').str.split(',')
    #df[col] = df[col].apply(my_function)
    
"""

    This function processes the text and gets rid off the extra spaces within the labels
    
    Input - Array of Strings
    
    Example - Input - ['  ArmedAssault', 'ArmedAssault  ']
              Output - ['ArmedAssault', 'ArmedAssault']
    
    

"""
def proceess_text(data):
    for i in range(len(data)):
        data[i] = ''.join(c for c in data[i] if c not in '"')
        data[i] = ''.join(c for c in data[i] if c not in '\'')
        data[i] = data[i].strip()

"""

    This function retrives the distinct labels from each row of a particular column
    
    INPUT - Column Values
    
    Example - Input - ['ArmedAssault', 'ArmedAssault']
              Output - ['ArmedAssault']

"""      
        
def get_unique(data):
    for i in range(len(data)):
        # print(data[i])
        proceess_text(data[i])
        data[i] = list(set(data[i]))
    
    
    

In [14]:
import itertools
from collections import defaultdict

"""
    
    This function takes an nested list of strings. After that it flattens the entire array. For example,
    [['kidnapped', 'killed'], ['injured']] becomes ['kidnapped', 'killed','injured']. The reason for doing
    this is that we dont want any duplicates. That is why we are getting rid off the redudancies. 
    
    This function also can produce the Top K labels of given column with its respective counts.

"""

def flatten_list(data, K):
    # get_unique(data)
    flatten = list(itertools.chain.from_iterable(data))
    #proceess_text(flatten)
    records = defaultdict(int)
    track = []
    for i in flatten:
        records[i] += 1
    for k, v in records.items():
        track.append((v, k))
    track.sort(key = lambda x : [-x[0]])
    if K > 0:
        track = track[0:K]
    keep = set()
    for i in track:
        keep.add(i[1])
    final_list = []
    for i in flatten:
        if i in keep:
            final_list.append(i)
    return final_list

In [15]:
# process - masteer
for i in EVENTS:
    process_column(df, i)
    get_unique(df[i])

/Users/shahreenshahjahanpsyche/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
df.head(5)
df = df[df['is_relevant'] == True]

In [24]:
ACTION_TYPE = ['Assassination','Hijacking','Kidnapping','Barricade Incident','Bombing/Explosion','Armed Assault','Unarmed Assault','Facility/Infrastructure Attack','Unknown']

def isActionPresent(data, action):
    for i in data:
        if i == action:
            return True
    
    return False
    
    
for action in ACTION_TYPE:
    df[action] = df['action_type'].apply(isActionPresent, args=(action,))

df.head(5)
    



/Users/shahreenshahjahanpsyche/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,id,is_relevant,target,source,action,action_type,location,date,country,year,...,source_group,Assassination,Hijacking,Kidnapping,Barricade Incident,Bombing/Explosion,Armed Assault,Unarmed Assault,Facility/Infrastructure Attack,Unknown
0,32355,True,"[Military Police, Rangers]","[Lashkar e Jhangvi LeJ, militants]","[killing, killed, gunned]",[Armed Assault],[Karachi],[December 6],pakistan,2016,...,"[Attackers, Lashkar e Jhangvi]",False,False,False,False,False,True,False,False,False
1,32398,True,"[SFs, Security Forces, civilians]",[NotSpecified],"[wounded, killed, killing, fighting]",[Unknown],[NotSpecified],[December 20],pakistan,2016,...,[Unknown Source],False,False,False,False,False,False,False,False,True
2,29171,True,"[Naga, Nagas]","[Manipur Naga Peopleâ€™s Front, MNPF]","[exploded, attack, bomb, explosion]",[Bombing/Explosion],[NotSpecified],[August 30],india,2016,...,[Manipur Naga Peoples Front],False,False,False,False,True,False,False,False,False
3,25445,True,"[foreigners, hostages, Artisan Restaurant]","[suicide fighters, suicide soldiers, attackers]","[injuring, killing, gun battle]",[Armed Assault],[Dhaka],"[August, 27th]",bangladesh,2016,...,[Attackers],False,False,False,False,False,True,False,False,False
4,31739,True,"[Muttahida Qaumi Movement, MQM]",[NotSpecified],[shot],[Assassination],"[Korangi, Sindh, Naeemabad, Karachi]",[June 19],pakistan,2016,...,[Unknown Source],True,False,False,False,False,False,False,False,False


In [25]:
COLUMNS = ['news', 'Assassination','Hijacking','Kidnapping','Barricade Incident','Bombing/Explosion','Armed Assault','Unarmed Assault','Facility/Infrastructure Attack','Unknown']


df_action = df.loc[:, COLUMNS]
df_action.head(5)


,news,Assassination,Hijacking,Kidnapping,Barricade Incident,Bombing/Explosion,Armed Assault,Unarmed Assault,Facility/Infrastructure Attack,Unknown
0,Two Lashkar e Jhangvi LeJ militants Asim alias...,False,False,False,False,False,True,False,False,False
1,The Commander Southern Command Lieutenant Gene...,False,False,False,False,False,False,False,False,True
2,Meanwhile the underground organization Manipur...,False,False,False,False,True,False,False,False,False
3,Islamic State IS in the latest issue of its on...,False,False,False,False,False,True,False,False,False
4,A senior Muttahida Qaumi Movement MQM worker i...,True,False,False,False,False,False,False,False,False


In [26]:
output_path = '/Users/shahreenshahjahanpsyche/Desktop/SATP-Event-Classification/Data_MachineLearning'

df_action.to_csv(output_path+'/action_type_data.csv', header = True, index = False)

In [27]:
df_action.shape

(727, 10)